In [ ]:
# Install necessary packages
!apt-get update && apt-get install -y openjdk-11 maven wget unzip

# Set environment variables
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
os.environ['HADOOP_HOME'] = '/content/hadoop'
os.environ['PATH'] += ':' + os.environ['HADOOP_HOME'] + '/bin:' + os.environ['HADOOP_HOME'] + '/sbin'

# Download and unpack Hadoop
!wget https://archive.apache.org/dist/hadoop/common/hadoop-3.3.5/hadoop-3.3.5.tar.gz
!tar -xvf hadoop-3.3.5.tar.gz
!mv hadoop-3.3.5 hadoop

# Download and unpack Giraph
!wget https://archive.apache.org/dist/giraph/giraph-1.3.0/giraph-1.3.0-for-hadoop-3-jar-with-dependencies.jar
# (Alternatively clone source and build with maven)


In [ ]:
# Configure minimal hadoop settings for local mode
!cat <<EOF > $HADOOP_HOME/etc/hadoop/core-site.xml
<configuration>
  <property>
    <name>fs.defaultFS</name>
    <value>hdfs://localhost:9000</value>
  </property>
</configuration>
EOF

!cat <<EOF > $HADOOP_HOME/etc/hadoop/hdfs-site.xml
<configuration>
  <property>
    <name>dfs.replication</name>
    <value>1</value>
  </property>
</configuration>
EOF

# Format namenode and start dfs & yarn
!$HADOOP_HOME/bin/hdfs namenode -format
!$HADOOP_HOME/sbin/start-dfs.sh
!$HADOOP_HOME/sbin/start-yarn.sh


In [ ]:
# Download the dataset
!wget https://snap.stanford.edu/data/wiki-Vote.txt.gz
!gunzip wiki-Vote.txt.gz

# Convert to adjacency list format for Giraph: each line “vertex_id neighbour1:0 neighbour2:0 …”
# Using python preprocessing
import sys
from collections import defaultdict

adj = defaultdict(list)
with open('wiki-Vote.txt','r') as f:
    for line in f:
        if line.startswith('#'):
            continue
        src, dst = line.strip().split()
        adj[src].append(dst)

with open('wikiVote_adj.txt','w') as f_out:
    for v, nbrs in adj.items():
        line = v + ' ' + ' '.join([nbr+':0' for nbr in nbrs])
        f_out.write(line + '\n')

# Upload that to HDFS
!$HADOOP_HOME/bin/hdfs dfs -mkdir /input
!$HADOOP_HOME/bin/hdfs dfs -put wikiVote_adj.txt /input/


In [ ]:
!git clone https://github.com/apache/giraph.git
!cd giraph && mvn clean install -DskipTests

# For simplicity, assume we’ll use pre-built jar:
GIRAPH_JAR = '/content/giraph-1.3.0-for-hadoop-3-jar-with-dependencies.jar'


In [ ]:
# Run the Weakly Connected Components job
!hadoop jar {GIRAPH_JAR} org.apache.giraph.GiraphRunner \
  org.apache.giraph.examples.WCCComputation \
  -vif org.apache.giraph.io.formats.LongLongNullTextInputFormat \
  -vip /input/wikiVote_adj.txt \
  -vof org.apache.giraph.io.formats.IdWithValueTextOutputFormat \
  -op /output/WCC -w 1

# Capture execution time (you may note output time from log)


In [ ]:
# Two‐phase SCC (implement your class) 
!hadoop jar {GIRAPH_JAR} your.package.SCCComputation \
  -vif … -vip /input/wikiVote_adj.txt -vof … -op /output/SCC -w 1


In [ ]:
# Run triangle counting – implement your class
!hadoop jar {GIRAPH_JAR} your.package.TriangleCountComputation \
  -vif … -vip /input/wikiVote_adj.txt -vof … -op /output/Triangles -w 1


In [ ]:
# Run clustering coefficient – implement your class
!hadoop jar {GIRAPH_JAR} your.package.ClusteringCoefficientComputation \
  -vif … -vip /input/wikiVote_adj.txt -vof … -op /output/ClusteringCoeff -w 1


In [ ]:
# Run BFS sampling based diameter job – implement your class
!hadoop jar {GIRAPH_JAR} your.package.DiameterComputation \
  -vif … -vip /input/wikiVote_adj.txt -vof … -op /output/Diameter -w 1